In [2]:
import pandas as pd
import numpy as np
import re
import ast


train_df = pd.read_csv('data/train.csv')
corpus_df = pd.read_csv('data/corpus.csv')

print(f"Kích thước tập Train ban đầu: {train_df.shape}")
print(f"Kích thước tập Corpus ban đầu: {corpus_df.shape}")
print("\nMẫu cid thô trong train:", train_df['cid'].iloc[7]) 


Kích thước tập Train ban đầu: (119456, 4)
Kích thước tập Corpus ban đầu: (261597, 2)

Mẫu cid thô trong train: [31682 31677]


In [3]:
def clean_cid(cid_str):
    if pd.isna(cid_str):
        return []
    
    cid_str = str(cid_str)

    numbers = re.findall(r'\d+', cid_str)

    return [int(num) for num in numbers]


train_df['cid_list'] = train_df['cid'].apply(clean_cid)


print("Sample sau khi xử lý:")
print(train_df[['cid', 'cid_list']].iloc[7]) # Xem dòng có 2 cid

Sample sau khi xử lý:
cid          [31682 31677]
cid_list    [31682, 31677]
Name: 7, dtype: object


In [4]:
valid_cids = set()

for cids in train_df['cid_list']:
    valid_cids.update(cids)

print(f"Số lượng documents (cid) duy nhất được tham chiếu trong tập Train: {len(valid_cids)}")

Số lượng documents (cid) duy nhất được tham chiếu trong tập Train: 68860


In [5]:
corpus_df['cid'] = pd.to_numeric(corpus_df['cid'], errors='coerce')


filtered_corpus_df = corpus_df[corpus_df['cid'].isin(valid_cids)].copy()

filtered_corpus_df.reset_index(drop=True, inplace=True)

print(f"Corpus sau khi lọc: {filtered_corpus_df.shape}")
print(f"Đã loại bỏ {len(corpus_df) - len(filtered_corpus_df)} documents không dùng đến.")

Corpus sau khi lọc: (68663, 2)
Đã loại bỏ 192934 documents không dùng đến.


In [6]:
# Lấy danh sách cid thực tế có trong corpus sau khi lọc
available_cids = set(filtered_corpus_df['cid'].unique())

# Tìm những cid có trong Train nhưng KHÔNG tìm thấy trong Corpus
missing_cids = valid_cids - available_cids

if len(missing_cids) > 0:
    print(f"⚠️ CẢNH BÁO: Có {len(missing_cids)} cid xuất hiện trong Train nhưng KHÔNG có nội dung trong Corpus.")
    print("Các cid bị thiếu (ví dụ 5 cái đầu):", list(missing_cids)[:5])
    # Tùy chiến lược, bạn có thể muốn loại bỏ các dòng train này
else:
    print("✅ Dữ liệu sạch: Tất cả cid trong Train đều tìm thấy nội dung trong Corpus.")

⚠️ CẢNH BÁO: Có 197 cid xuất hiện trong Train nhưng KHÔNG có nội dung trong Corpus.
Các cid bị thiếu (ví dụ 5 cái đầu): [88577, 94722, 164867, 100867, 110081]


In [7]:
# Lưu lại để dùng cho bước Chunking/Embedding tiếp theo
filtered_corpus_df.to_csv('filtered_corpus.csv', index=False)
train_df.to_csv('processed_train.csv', index=False) # File này đã có cột cid_list dạng list thật

print("Đã lưu filtered_corpus.csv và processed_train.csv")

Đã lưu filtered_corpus.csv và processed_train.csv
